在Baseline的单资产DLinear模型上，预测结果取得了稳定的超额准确率，但是也会出现几个问题

1. 由于单资产的数据量有限，导致训练集规模不够大， 容易出现过拟合现象，导致学习不充分

2. 由于模型需要在“预测结果本身是Neutral分类”和“对当前样本做出的预测不够有信心” 两种情况下都要选择放弃预测，从而导致整体预测结果会出现比真实结果更多的Neutral 分类（约90%），模型仅对10%的样本选择做出预测。

为了能提高资金利用效率，防止模型在大多数情况下都在被动等待，我们可以在单资产DLinear Model的基础上，扩展资产的数量，一方面增大数据集，另一方面降低资金利用效率。

在这一阶段，为了防止模型过于复杂，我们暂时还不考虑引入跨资产的截面数据关系，假定这些资产之间是完全无关的，这也符合PatchTST中通道独立的假设，

除了上证50、沪深300、中证500股指期货以外，我们选取了一些成交量好，与宏观经济有强关联的资产种类，例如黄金、原油、焦煤、螺纹钢、热卷、铁矿石、豆粕、棉花

但是由于不同的期货品种共用模型参数，而资产的波动率、行为模式存在差异，导致模型的预测能力下降了。

对此，我们可以有两个解决方法

1. 两阶段训练模式，先训练整体，再根据目标微调参数

2. 增加资产信息维度，将资产信息嵌入到价格数据中，让模型学会根据资产信息改变参数

我们先实现第一种思路

In [1]:
import os
os.chdir('d:/future/index_future_prediction/Index_Future_Prediction')

In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import lr_scheduler, Adam, AdamW
from scipy.stats import norm, t

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
from utils.random_split import RandomSplit, CallableDataset
from utils.back_test import BackTest
from utils.hybrid_loss import HybridLoss
from utils.hybrid_decoder import HybridDecoder
from utils.prediction_recorder import PredictionRecorder
from utils.train_animator import TrainAnimator
from utils.model_train import ModelTrain
from utils.get_ohlcv import GetOHLCV

In [4]:
import tushare as ts
pro = ts.pro_api('700c1d6015ad030ff20bf310c088243da030e6b79a2a1098d58d2614')

In [5]:
from modules.dlinear import DLinear
class DLinearOutput(nn.Module):
    """DLinear作为信息编码层，编码历史信息，然后交给HybridDecoder进行解码"""
    def __init__(self, seq_len, pred_len, individual, enc_in, kernel_size, init_prob, dropout, **kwargs):
        super(DLinearOutput, self).__init__(**kwargs)
        self.device = 'cuda:0'

        self.process = nn.Sequential(
            DLinear(seq_len = seq_len, pred_len =  pred_len, individual = individual, enc_in = enc_in, kernel_size = kernel_size),
            nn.Flatten(start_dim = 1), 
            nn.Dropout(dropout),
            )


        self.output = HybridDecoder(pred_len * enc_in, init_prob = init_prob)

    def forward(self, x):
        x = self.process(x)
        return self.output(x)

In [6]:
assets_list = ['IH.CFX', 'IF.CFX', 'IC.CFX', 'AU.SHF', 'FU.SHF', 'JM.DCE','RB.SHF','HC.SHF', 'I.DCE', 'M.DCE', 'CF.ZCE',]

In [7]:
seq_len = 40
pred_len = 5
train_ratio = 0.5
validation_ratio = 0.2
test_ratio = 0.02
threshold_ratio = 0.25

In [8]:
def get_random_split(train_ratio, validation_ratio, test_ratio):
    source = GetOHLCV()
    sample_date = source.get_data('M.DCE', 5, 0.3)
    date_column = sample_date['trade_date'].copy()
    total_size = len(date_column)
    train_size = int(train_ratio * total_size)
    validation_size = int(validation_ratio * total_size)
    test_size = int(test_ratio * total_size)
    random_split = np.random.randint(train_size, total_size - validation_size - test_size)
    validation_start = date_column.iloc[random_split]
    test_start = date_column.iloc[random_split+validation_size]
    test_end = date_column.iloc[random_split+validation_size+test_size]
    return validation_start, test_start, test_end

In [9]:
def get_data_set(assets_list, validation_start, test_start, test_end, seq_len, pred_len, threshold_ratio):

    source = GetOHLCV()

    train_set = None
    validation_set = None
    test_set = None
    feature_column = ['log_open','log_high','log_low','log_close','log_amount']
    label_column = ['label_return','down_prob','middle_prob','up_prob']
    
    for code in assets_list:

        data = source.get_data(code, pred_len, threshold_ratio)

        train_data = data[data['trade_date'] < validation_start].copy()
        validation_data = data[(data['trade_date'] >= validation_start) & (data['trade_date'] < test_start)].copy()
        test_data = data[(data['trade_date'] >= test_start) & (data['trade_date'] < test_end)].copy()
    
        train_feature = torch.tensor(train_data[feature_column].values, dtype = torch.float32, device = 'cuda:0')
        train_feature = train_feature.unfold(dimension = 0, size = seq_len, step = 1).transpose(1,2)

        validation_feature = torch.tensor(validation_data[feature_column].values, dtype = torch.float32, device = 'cuda:0')
        validation_feature = validation_feature.unfold(dimension = 0, size = seq_len, step = 1).transpose(1,2)

        test_feature = torch.tensor(test_data[feature_column].values, dtype = torch.float32, device = 'cuda:0')
        test_feature = test_feature.unfold(dimension = 0, size = seq_len, step = 1).transpose(1,2)



        train_label = torch.tensor(train_data[label_column].values, dtype = torch.float32, device = 'cuda:0')
        train_label = train_label[seq_len-1:]

        validation_label = torch.tensor(validation_data[label_column].values, dtype = torch.float32, device = 'cuda:0')
        validation_label = validation_label[seq_len-1:]

        test_label = torch.tensor(test_data[label_column].values, dtype = torch.float32, device = 'cuda:0')
        test_label = test_label[seq_len-1:]



        if train_set == None:
            train_set = CallableDataset(train_feature, train_label)
        else:
            train_set = train_set + CallableDataset(train_feature, train_label)

        if validation_set == None:
            validation_set = CallableDataset(validation_feature, validation_label)
        else:
            validation_set = validation_set + CallableDataset(validation_feature, validation_label)
        
        if test_set == None:
            test_set = CallableDataset(test_feature, test_label)
        else:
            test_set = test_set + CallableDataset(test_feature, test_label)

    return train_set, validation_set, test_set

In [10]:
validation_start, test_start, test_end = get_random_split(train_ratio, validation_ratio, test_ratio)

In [11]:
train_set, validation_set, test_set = get_data_set(assets_list, validation_start, test_start, test_end, seq_len, pred_len, threshold_ratio)

In [12]:
recorder = PredictionRecorder()
animator = TrainAnimator(figsize=(12,6))

Animator data has been reset.


实现两阶段的训练，由于设置了学习率调度器，在一阶段结束的时候学习率已经被调度到了一个相当低的水平，就不需要重新设置学习率了

In [ ]:
result = np.zeros(shape = (10, len(assets_list), 4))

for i in range(10):
    validation_start, test_start, test_end = get_random_split(train_ratio, validation_ratio, test_ratio)
    train_set, validation_set, test_set = get_data_set(assets_list, validation_start, test_start, test_end, seq_len, pred_len, threshold_ratio)

    for j in range(len(assets_list)):
        code = assets_list[j]
        train_set_2, validation_set_2, test_set_2 = get_data_set([code], validation_start, test_start, test_end, seq_len, pred_len, threshold_ratio)

        model = DLinearOutput(seq_len = seq_len, pred_len = pred_len, individual = True, enc_in = 5, kernel_size = 21, init_prob = [0.0, 1, 0.0], dropout = 0.5).to('cuda:0')
        loss_fn = HybridLoss(alpha = 1e-3, delta = 1)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay = 1e-1)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
        train = ModelTrain(model = model,
                        batch_size = 100,
                        train_set = train_set,
                        validation_set = validation_set,
                        test_set = test_set,
                        loss_fn = loss_fn,
                        optimizer = optimizer,
                        scheduler=scheduler,
                        recorder=recorder,
                        graph=animator,
                        )
        
        train.train_set = train_set_2
        train.validation_set = validation_set_2

        prediction, precision = train.epoch_train(epochs = 10, round = 100, early_stop = 10)
        result[i,j,0] = prediction
        result[i,j,1] = precision

        # prediction, precision = train.epoch_train(epochs = 10, round = 100, early_stop = 10)
        # result[i,j,2] = prediction
        # result[i,j,3] = precision


In [ ]:
all_assets = pd.DataFrame({
    'stage_1_prediction': np.mean(result, axis = 0)[:,0],
    'stage_2_prediction': np.mean(result, axis = 0)[:,2],

    'stage_1_precision': np.mean(result, axis = 0)[:,1],
    'stage_2_precision': np.mean(result, axis = 0)[:,3],

    'stage_1_precision_std': np.std(result, axis = 0)[:,1],
    'stage_2_precision_std': np.std(result, axis = 0)[:,3],

})

In [ ]:
all_assets_styled = all_assets.style.format({
    'stage_1_prediction': '{:.1%}',
    'stage_1_precision': '{:.1%}',
    'stage_1_precision_std': '{:.1%}',
    'stage_2_prediction': '{:.1%}',
    'stage_2_precision': '{:.1%}',
    'stage_2_precision_std': '{:.1%}',
})
all_assets_styled

In [ ]:
from IPython.display import display, Markdown
display(Markdown(all_assets.to_markdown(index=False)))